In [13]:
import sqlite3
import pandas as pd
import numpy as np
import time
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.moses import MosesDetokenizer

from stop_words import get_stop_words
from collections import defaultdict
import gensim
from gensim import corpora, models, similarities
from gensim.models.doc2vec import TaggedDocument,TaggedLineDocument
from gensim.models import Doc2Vec
import gensim.models.doc2vec



# #from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
# from collections import OrderedDict
# import multiprocessing
# from IPython.display import Image
# import requests
# from random import randint 
# from sklearn.manifold import TSNE
# from sklearn.cluster import AffinityPropagation
# #import matplotlib.pyplot as plt
# #%matplotlib inline

## Full Body Model Training

In [14]:
conn = sqlite3.connect("/volume/testDB.db")

In [22]:
df = pd.read_sql('SELECT doi,abstract,body from PLOS_ALL', conn)

In [23]:
df.head()

,doi,abstract,body
0,10.1371/journal.pone.0143437,Bone scintigraphies are widely applied for det...,Pulmonary and prostate cancers are the most co...
1,10.1371/journal.pone.0096769,Cystic fibrosis common autosomal recessive d...,"Voice disorders, or dysphonias, can be caused ..."
2,10.1371/journal.pone.0177044,Alzheimer’ disease neurodegenerative syndrom...,1. Dementia is typically defined as a clinical...
3,10.1371/journal.pone.0033200,PurposeXeroderma pigmentosum group (XPD) code...,Non-small cell lung cancer (NSCLC) accounts fo...
4,10.1371/journal.pone.0001814,Class MHC molecules display peptides the cel...,The endosomal route is considered to be the de...


In [5]:
ab_tok = pd.read_pickle('/volume/abtokens.pkl')

In [6]:
texts = ab_tok.tolist()

In [15]:
tdf = pd.read_sql('SELECT * FROM PLOS_all_tok', conn)

In [16]:
tdf.head()

,level_0,index,doi,bod_toks
0,0,0,10.1371/journal.pone.0143437,"['pulmonari', 'prostat', 'cancer', 'common', '..."
1,1,1,10.1371/journal.pone.0096769,"['voic', 'disord', 'dysphonia', 'can', 'caus',..."
2,2,2,10.1371/journal.pone.0177044,"['dementia', 'typic', 'defin', 'clinic', 'synd..."
3,3,3,10.1371/journal.pone.0033200,"['non', 'small', 'cell', 'lung', 'cancer', 'ns..."
4,4,4,10.1371/journal.pone.0001814,"['endosom', 'rout', 'consid', 'default', 'path..."


In [17]:
bodtexts = tdf['bod_toks'].tolist()

In [32]:
tok = bodtexts[1].split(', ')

In [40]:
tnew = []
for line in tok:
    l = line.replace("'","")
    l = l.strip('[]')
    tnew.append(l)

In [18]:
bods = []

for i in range(0,len(bodtexts)):
    det = bodtexts[i].replace(',','')
    det = det.replace("'","")
    det = det.strip('[]')
    bods.append(det)
    if i % 20000 == 0:
        print(i)

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000


In [ ]:
bods[1]

#### toklen

In [ ]:
tok_len = []

for i in range(0,len(bods)):
    tok_len.append(len(bods[i].split()))
    if i % 10000 == 0:
        print(i)
df = pd.DataFrame(tok_len)
df.describe()

In [32]:
df.columns = ['toklen']

In [34]:
df[df['toklen'] > 10000].count() 

toklen    449
dtype: int64

In [35]:
df[df['toklen'] > 10000].describe()

,toklen
count,449.000000
mean,11786.195991
std,2063.130425
min,10001.000000
25%,10523.000000
50%,11114.000000
75%,12338.000000
max,27798.000000


In [26]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [29]:
corpora.MmCorpus.serialize('/volume/models/corpus.mm', corpus)
dictionary.save('/volume/models/dictionary.dict')

In [30]:
tfidf = models.TfidfModel(corpus, normalize=True)
tfidf.save('/volume/model.tfidf')

In [31]:
start_time = time.time()
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=200)
lsi.save('/volume/models/model.lsi')
print("--- %s seconds ---" % (time.time() - start_time))
#corpus_lsi = lsi[corpus_tfidf]

In [32]:
start_time = time.time()
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=200)
lda.save('/volume/models/model.lda')
print("--- %s seconds ---" % (time.time() - start_time))
#corpus_lda = lda[corpus_tfidf]

In [ ]:
corpus_tfidf = tfidf[corpus]
corpus_lsi = lsi[corpus_tfidf]
corpus_lda = lda[corpus_tfidf]

In [10]:
index = similarities.MatrixSimilarity(lsi[corpus])
index.save('/volume/models/similarity.index')

In [9]:
file = open('/volume/processed_body_docs.txt', 'w')

for i in range(0,len(bods)):
    detok_str = bods[i]
    file.write("%s\n" % detok_str)

In [3]:
documents = []
with open('/volume/processed_body_docs.txt') as f:
    for line in f:
        documents.append(TaggedLineDocument(line))

In [9]:
type(documents[1])

gensim.models.doc2vec.TaggedLineDocument

In [5]:
# detok for abstracts 

detokenizer = MosesDetokenizer()
file = open('/volume/processed_docs.txt', 'w')

for i in range(0,len(texts)):
    detok_str = detokenizer.detokenize(texts[i], return_str=True)
    file.write("%s\n" % detok_str)

In [26]:
bods_sm = bods[0:10]

In [27]:
len(bods_sm)

10

In [28]:
#documents = [doc for doc in TaggedLineDocument('/volume/processed_docs.txt')]
documents = [doc for doc in TaggedLineDocument(bods_sm)]


TypeError: don't know how to handle uri ['pulmonari prostat cancer common malign tumor often develop bone metastas tumor cell spread bone mainli hematogen metastasi howev mechan underli bone metastasi pulmonari prostat cancer differ specif pulmonari cancer cell metastas bone mainli pulmonari vein wherea prostat cancer cell spread bone mainli via vertebr venou system differ rout metastasi can produc differ distribut featur bone metastas although studi morgan et al dodd et al support viewpoint bone scintigraphi import techniqu detect bone metastas evalu stage malign tumor prognosi patient well assess tumor respons therapi effect therapeut schedul tool can use scan entir skeleton patient therebi allow bone lesion display entireti thu bone scintigraphi use studi distribut featur bone metastas analyz relationship pathway tumor spread distribut metastat lesion numer studi bone metastas pulmonari prostat cancer compar statist analys mani studi shown predilect site bone metastas pulmonari pros

In [ ]:
model = Doc2Vec(documents, size=200, window=5, min_count=5,workers=15, iter=10)

np.save('models/body_features-w2v-200.npy',model.docvecs.doctag_syn0)
model.save('models/body_features-w2v-200.doc2vec')